In [2]:
import pandas as pd

# Lire le CSV
df = pd.read_csv("offre_pathologie.csv")

# Afficher les noms de colonnes
print(df.columns)

# Afficher un aperçu des premières lignes
print(df.head())

Index(['famille_pathologie', 'specialite', 'geopoint medecin'], dtype='object')
  famille_pathologie specialite           geopoint medecin
0  Cancers / Tumeurs  Oncologie   POINT(2.32706 48.878401)
1  Cancers / Tumeurs  Oncologie  POINT(2.273756 48.839637)
2  Cancers / Tumeurs  Oncologie  POINT(2.273756 48.839637)
3  Cancers / Tumeurs  Oncologie  POINT(-2.26804 47.261792)
4  Cancers / Tumeurs  Oncologie  POINT(1.676487 48.998049)


### LIRE LES FICHIERS

In [4]:
import geopandas as gpd
import pandas as pd
from shapely import wkt

# Clusters IRIS
gdf_clusters = gpd.read_file("iris_clusters_200.gpkg")  # contient cluster_id et geometry

# Offre médicale
df_offre = pd.read_csv("offre_pathologie.csv")  # colonnes: Famille Pathologie, Spécialité, geopoint


In [6]:
# Convertir WKT en geometry
df_offre['geometry'] = df_offre['geopoint medecin'].apply(wkt.loads)

# Transformer en GeoDataFrame
gdf_offre = gpd.GeoDataFrame(df_offre, geometry='geometry', crs="EPSG:4326")

## CREATION DES CLUSTERS

In [7]:
gdf_offre_in_cluster = gpd.sjoin(
    gdf_offre,
    gdf_clusters[['cluster_id', 'geometry']],
    how="left",
    predicate="within"
)

# Filtrer les points hors cluster si nécessaire
gdf_offre_in_cluster = gdf_offre_in_cluster.dropna(subset=['cluster_id'])

In [8]:
offre_count_by_cluster = gdf_offre_in_cluster.groupby('cluster_id').size().reset_index(name='offre_count')

In [10]:
gdf_offre_viz = gdf_offre_in_cluster.merge(
    offre_count_by_cluster,
    on='cluster_id',
    how='left'
)

# Colonnes utiles pour visualisation
gdf_offre_viz = gdf_offre_viz[['famille_pathologie', 'specialite', 'cluster_id', 'offre_count', 'geometry']]

In [11]:
# GeoPackage pour carte SIG / Dataiku
gdf_offre_viz.to_file("offre_pathologie_clusters.gpkg", driver="GPKG")

# CSV pour DataViz classique (longitude / latitude séparées)
gdf_offre_viz['longitude'] = gdf_offre_viz.geometry.x
gdf_offre_viz['latitude'] = gdf_offre_viz.geometry.y
gdf_offre_viz.drop(columns='geometry').to_csv("offre_pathologie_clusters.csv", index=False)

## ANNEXE LIAISON OFFRE PAR DEMANDE

In [13]:
df = pd.read_csv("pathologie par departement.csv")

# Afficher les noms de colonnes
print(df.columns)

Index(['ZONE', 'ZONE_0', 'departement', 'FAMILLE_PATHOLOGIE', 'ind_freq_sum'], dtype='object')
